# Analyze and plot methods for hurricane forecast uncertainty
##### author: Elizabeth A. Barnes, Randal J. Barnes and Mark DeMaria

## Python setup

In [ ]:
import datetime
import os
import pickle
import pprint
import time

import experiment_settings
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import shash
from build_data import build_hurricane_data
import build_model
import model_diagnostics
from silence_tensorflow import silence_tensorflow
import prediction
from sklearn.neighbors import KernelDensity
import pandas as pd
from tqdm import tqdm
import analysis_plots
from scipy import stats
from toolbox import custom_round, ceiling_round
import seaborn as sns

import imp
import warnings
warnings.filterwarnings("ignore")
silence_tensorflow()
dpiFig = 400

mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["figure.dpi"] = 150
np.warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

## User parameters

In [ ]:
__author__  = "Randal J Barnes and Elizabeth A. Barnes"
__version__ = "17 March 2022"

EXP_NAME_LIST = (
    # "intensity1_AL48",
    # "intensity2_AL72",
    # "intensity3_AL96",    
    # "intensity4_EPCP48",
    "intensity5_EPCP24",    
    # "intensity5_EPCP72",
    # "intensity6_EPCP96",       
)

PLOT_SEED_VECTOR = {"intensity4_EPCP48": 739,
                    "intensity5_EPCP24": 739,
                    "intensity1_AL48": None,
                    "intensity2_AL72": None,                    
                    "intensity3_AL96": None,                                        
                   }
TESTING_YEAR_VECTOR = {"intensity4_EPCP48": 2020,
                       "intensity5_EPCP24": 2020,
                      }
                        
    

SHASH_INCS = np.arange(-160,161,1)
DATA_PATH = "data/"
MODEL_PATH = "saved_models/"
FIGURE_PATH = "figures/paper_figures/"

In [ ]:
RI_THRESH_DICT = {24: 30,
                  48: 55,
                  72: 65,
                 }

## Plotting functions

In [ ]:
FS = 16
colors = ('#284E60','#E1A730','#D95980','#C3B1E1','#351F27','#A9C961')

### for white background...
plt.rc('text',usetex=True)
plt.rc('font',**{'family':'sans-serif','sans-serif':['Avant Garde']}) 
plt.rc('savefig',facecolor='white')
plt.rc('axes',facecolor='white')
plt.rc('axes',labelcolor='dimgrey')
plt.rc('axes',labelcolor='dimgrey')
plt.rc('xtick',color='dimgrey')
plt.rc('ytick',color='dimgrey')
################################  
################################  
def adjust_spines(ax, spines):
    for loc, spine in ax.spines.items():
        if loc in spines:
            spine.set_position(('outward', 5))
        else:
            spine.set_color('none')  
    if 'left' in spines:
        ax.yaxis.set_ticks_position('left')
    else:
        ax.yaxis.set_ticks([])
    if 'bottom' in spines:
        ax.xaxis.set_ticks_position('bottom')
    else:
        ax.xaxis.set_ticks([]) 

def format_spines(ax):
    adjust_spines(ax, ['left', 'bottom'])
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    ax.spines['left'].set_color('dimgrey')
    ax.spines['bottom'].set_color('dimgrey')
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    ax.tick_params('both',length=4,width=2,which='major',color='dimgrey')
#     ax.yaxis.grid(zorder=1,color='dimgrey',alpha=0.35)    
    

In [ ]:
def get_storm_details(df, isample):
    storm = df.iloc[isample]
    storm_name = storm['Name']
    storm_ftime = storm['ftime(hr)']
    storm_month = str(storm['time'])[:-4]
    storm_day = str(storm['time'])[-4:-2]
    storm_hour = str(storm['time'])[-2:]
    storm_year = storm['year']

    details = storm_name + ' ' + str(storm_year) + '-' + str(storm_month) + '-' + str(storm_day) + ' ' + str(storm_hour) + '00 @' + str(storm_ftime) + 'hr'
    
    return details

In [ ]:
def plot_iqr_vs_error(ax,model, x_eval, onehot_eval, df_eval, yvar="shash_error"):
    # make predictions
    mu_pred, sigma_pred, gamma_pred, tau_pred = prediction.params(x_eval, model)    

    # convert y_pred to intensity corrections    
    ann_correction  = prediction.percentile_value(mu_pred,sigma_pred,gamma_pred,tau_pred,percentile_frac=.5)
    true_correction = df_eval['OBDV'].to_numpy()  

    # convert to full intensity predictions
    cons_intensity  = df_eval['VMXC'].to_numpy()
    ann_intensity   = cons_intensity + ann_correction
    true_intensity  = cons_intensity + true_correction

    # compute errors
    shash_error  = np.abs(ann_intensity - true_intensity)
    cons_error = np.abs(cons_intensity - true_intensity)

    # covert shash prediction bounds
    shash_low_correction  = prediction.percentile_value(mu_pred,sigma_pred,gamma_pred,tau_pred,percentile_frac=.25)
    shash_high_correction = prediction.percentile_value(mu_pred,sigma_pred,gamma_pred,tau_pred,percentile_frac=.75)
    iqr                   = shash_high_correction - shash_low_correction

    print(stats.spearmanr(iqr,shash_error))
    print(stats.pearsonr(iqr,shash_error))

    df_eval['iqr'] = iqr
    df_eval['shash_error'] = shash_error
    df_eval['cons_error'] = cons_error

    #------------------------------------------------------------------------------------
    ROUND_BASE = 10

    df_plot = df_eval.copy()
    df_plot['iqr'] = pd.Series(df_plot['iqr']).apply(lambda x: custom_round(x, base=ROUND_BASE))
    df_plot['iqr'] = pd.Series(df_plot['iqr']).apply(lambda x: ceiling_round(x, ceiling=30))
    #-----------------------------------------------------------------------------
    clr = colors[0]
    if(yvar=="cons_error"):
        clr =colors[1]
    
    for xvar in ('iqr',):#('sigma', 'range_66'):
        g = sns.boxplot(
            ax=ax,
            data=df_plot,
            x=xvar, y=yvar,
            whis=None,
            fliersize=0,
            boxprops={'alpha':.2,
                      'edgecolor': 'gray',
                      'color': 'gray',
                     },
        )      
        plt.setp(g.artists, edgecolor = '.4', facecolor='.95')
        plt.setp(g.lines, color='.4')
        ax = sns.stripplot(ax=ax,x=xvar, y=yvar, data=df_plot, color=clr, size=3.,alpha=.75,)
        # plt.axhline(y=0,color='k', alpha=.25, linewidth=1)

        # g.set_xlim(-.5,6.5)

        ax.set_xlabel('ANN Predicted IQR [knots]')
        if(yvar=="cons_error"):
            ax.set_title('(c) Consensus Error vs ANN Predicted Uncertainty', fontsize=FS, color='k')    
            ax.set_ylabel('Consensus Absolute Error [knots]')
        else:
            ax.set_title('(b) ANN Error vs ANN Predicted Uncertainty', fontsize=FS, color='k')            
            ax.set_ylabel('ANN Absolute Error [knots]')        



## Network functions

In [ ]:
def load_model():
    model_name = (
        exp_name + "_" + 
        str(TESTING_YEARS) + '_' +
        settings["uncertainty_type"] + '_' + 
        f"network_seed_{network_seed}_rng_seed_{settings['rng_seed']}"
    )
    pprint.pprint(model_name)
    
    if settings["uncertainty_type"] == "bnn":       
        model = build_model.build_bnn_model(
            x_train,
            onehot_train,
            hiddens=settings["hiddens"],
            output_shape=onehot_train.shape[1],
            act_fun=settings["act_fun"],
        )
        
    elif settings["uncertainty_type"] == "mcdrop":       
        model = build_model.build_mcdrop_model(
            x_train,
            onehot_train,
            dropout_rate=settings["dropout_rate"],                
            hiddens=settings["hiddens"],
            output_shape=onehot_train.shape[1],
            act_fun=settings["act_fun"],
        )

    elif settings["uncertainty_type"][:5] == "shash": 
        model = build_model.build_shash_model(
            x_train,
            onehot_train,
            hiddens=settings["hiddens"],
            output_shape=onehot_train.shape[1],
            ridge_penalty=settings["ridge_param"],
            act_fun=settings["act_fun"],
        )
        
    model.load_weights(MODEL_PATH + model_name + "_weights.h5")
        
    return model, model_name

In [ ]:
def get_predictions(model, x_eval):
    runs = 5_000
    shash_cpd = np.zeros((np.shape(x_eval)[0],len(SHASH_INCS)))
    shash_med = np.zeros((np.shape(x_eval)[0],))
    mc_cpd = np.zeros((np.shape(x_eval)[0],runs))
    tf.random.set_seed(network_seed)

    if settings["uncertainty_type"] == "bnn":               
        for i in tqdm(range(0,runs)):
            mc_cpd[:,i] = np.reshape(model.predict(x_eval),np.shape(mc_cpd)[0])
        return mc_cpd, np.median(mc_cpd,axis=1)
    
    elif settings["uncertainty_type"] == "mcdrop":  
        # loop through runs for mcdrop calculation  
        for i in tqdm(range(0,runs)):
            mc_cpd[:,i] = np.reshape(model(x_eval,training=True),np.shape(mc_cpd)[0])    
        return mc_cpd, np.median(mc_cpd,axis=1)
    
    elif settings["uncertainty_type"][:5] == "shash": 
        # loop through samples for shash calculation and get PDF for each sample
        for j in tqdm(range(0,np.shape(shash_cpd)[0])):
            mu_pred, sigma_pred, gamma_pred, tau_pred = prediction.params( x_eval[np.newaxis,j], model )
            shash_cpd[j,:] = shash.prob(SHASH_INCS, mu_pred, sigma_pred, gamma_pred, tau_pred)    
            shash_med[j]   = shash.median(mu_pred,sigma_pred,gamma_pred,tau_pred)
        return shash_cpd, shash_med
    
    else:
        raise NotImplementedError('no such uncertainty type')


## Load the models

In [ ]:
pred_dict = {}
for exp_name in EXP_NAME_LIST:
    settings = experiment_settings.get_settings(exp_name)

    TESTING_YEARS = TESTING_YEAR_VECTOR[exp_name]
    settings["years_test"] = (TESTING_YEARS,)
    
    RNG_SEED = PLOT_SEED_VECTOR[exp_name]
    settings['rng_seed'] = RNG_SEED
    seed_dict = {}
    
    NETWORK_SEED_LIST = [settings["rng_seed"]]
    network_seed = NETWORK_SEED_LIST[0]
    tf.random.set_seed(network_seed)  # This sets the global random seed.  

    # get the data
    (
        data_summary,        
        x_train,
        onehot_train,
        x_val,
        onehot_val,
        x_test,
        onehot_test,        
        x_valtest,
        onehot_valtest,
        df_train,
        df_val,
        df_test,
        df_valtest,
    ) = build_hurricane_data(DATA_PATH, settings, verbose=0)
    # x_eval, onehot_eval, df_eval = x_test, onehot_test, df_test

    # load the correct model
    model, model_name = load_model()

    # get predictions
    pred_test = get_predictions(model, x_test)
    pred_valtest = get_predictions(model, x_valtest)

    seed_dict[network_seed] = {'pred_cpd_test': pred_test[0], 
                               'pred_med_test': pred_test[1],
                               'pred_cpd_valtest': pred_valtest[0], 
                               'pred_med_valtest': pred_valtest[1], 
                              }    
    pred_dict[exp_name] = seed_dict

print(pred_dict.keys())
# pred_dict['paper1_EPCP96'][605]['pred_cpd']

## Plot single network results

In [ ]:
# for i in np.arange(0,len(colors)):
#     plt.plot(i,i,'.',color=colors[i])
# plt.show()    

In [ ]:
PLOT_DATA = '_valtest'
#------------------------------------------

if PLOT_DATA == '_valtest':
    x_eval = x_valtest
    onehot_eval = onehot_valtest
    df_eval = df_valtest
    pred_eval = pred_dict[exp_name][RNG_SEED]['pred_cpd' + PLOT_DATA]    
elif PLOT_DATA == '_test':
    x_eval = x_test
    onehot_eval = onehot_test
    df_eval = df_test    
    pred_eval = pred_dict[exp_name][RNG_SEED]['pred_cpd' + PLOT_DATA]    
else:
    raise NotImplementedError('no such PLOT_DATA')

imp.reload(analysis_plots)
f, axs = plt.subplots(1, 3, figsize=(15/2*3,5))

# plot sample
ax1 = axs[1]
plot_iqr_vs_error(ax1,model, x_eval, onehot_eval, df_eval, yvar="shash_error")
ax1.set_ylim(0,65)
ax1.set_yticks(np.arange(0,70,10),np.arange(0,70,10))
ax1.set_xticks((0,1,2,3))
ax1.set_xticklabels(['0-10','10-20','20-30','30+'])    
format_spines(ax1)

ax2 = axs[2]
plot_iqr_vs_error(ax2,model, x_eval, onehot_eval, df_eval, yvar="cons_error")
ax2.set_ylim(0,65)
ax2.set_yticks(np.arange(0,70,10),np.arange(0,70,10))
ax2.set_xticks((0,1,2,3))
ax2.set_xticklabels(['0-10','10-20','20-30','30+'])    
format_spines(ax2)

# plot PITS
ax = axs[0]
analysis_plots.plot_pits(ax, x_eval, onehot_eval, model, pred_eval, None)
plt.ylim(0,.2)
plt.title('(a) PIT histogram comparison', fontsize=FS, color='k')
format_spines(ax)

plt.tight_layout()
ax.get_legend().remove()
# plt.savefig(FIGURE_PATH + 'pit_error_iqr' + exp_name + '_seed' + str(PLOT_SEED) + '.png', dpi=dpiFig)    
plt.show()

In [ ]:
PLOT_DATA = '_test'
from matplotlib import cm
#------------------------------------------

if PLOT_DATA == '_valtest':
    x_eval = x_valtest
    onehot_eval = onehot_valtest
    df_eval = df_valtest
    pred_eval = pred_dict[exp_name][RNG_SEED]['pred_cpd' + PLOT_DATA]    
    pred_med_eval = pred_dict[exp_name][RNG_SEED]['pred_med' + PLOT_DATA]    
elif PLOT_DATA == '_test':
    x_eval = x_test
    onehot_eval = onehot_test
    df_eval = df_test    
    pred_eval = pred_dict[exp_name][RNG_SEED]['pred_cpd' + PLOT_DATA][::1] 
    pred_med_eval = pred_dict[exp_name][RNG_SEED]['pred_med' + PLOT_DATA][::1]   
else:
    raise NotImplementedError('no such PLOT_DATA')
    
#------------------------------------------    
obs_dev_cons_hist, OBS_DEV_BINS = model_diagnostics.compute_clim_errors(
    onehot=np.append(onehot_train[:,0],onehot_val[:,0]), 
    smooth = False,
)

#------------------------------------------    
N_COLORS = 50
cmap_interp = cm.get_cmap('twilight_shifted', N_COLORS)    
value_remap = np.linspace(-15, 15, N_COLORS)
norm = mpl.colors.Normalize(vmin=value_remap.min(), vmax=value_remap.max())
    
plt.figure()
for p in np.arange(0,pred_eval.shape[0]):
    
    plt.plot(SHASH_INCS, pred_eval[p,:], 
             linewidth=1.5,
             color=cmap_interp(norm(pred_med_eval[p])),
             alpha=.5,
            )
plt.step(OBS_DEV_BINS,
         obs_dev_cons_hist, 
         linewidth=4, 
         color='k',
         alpha=.85,
         label='climatology',
        )    
yticks = np.arange(0,.22,.02)    
plt.yticks(yticks,yticks)    
plt.ylim(0,.18)

plt.legend(loc=2)

xticks = np.arange(-100,120,20)
plt.xticks(xticks,xticks)    
plt.xlim(-65,65)

plt.xlabel('predicted deviation from consensus (knots)')
plt.ylabel('probability density')

plt.title('(a) Predicted Distributions', color='k', fontsize=FS)

ax = plt.gca()
format_spines(ax)

# plt.savefig(FIGURE_PATH + 'predicted_distributions_' + model_name + '.png', dpi=dpiFig)    
plt.show()


## Plot examples

In [ ]:
# PLOTTING TESTING DATA ONLY
imp.reload(analysis_plots)
panel_letters = ('(b)','(c)','(d)','(e)')

f, axs = plt.subplots(2, 2, figsize=(10,8))
axs = axs.flatten()
# random_samples = (173,140,6,130) #AL @72
random_samples = (25,88,111,27) # (30,88,110,44)(25,88,110,27)

true_intensity = df_test["VMXC"] + df_test["OBDV"]
cons_intensity = df_test["VMXC"]

for isample, sample in enumerate(random_samples):
    
    cons_error = 0.0-onehot_test[sample,0]
    ann_error = pred_dict[exp_name][RNG_SEED]['pred_med_test'][sample]-onehot_test[sample,0]
    print(ann_error,cons_error )   
    
    ax = axs[isample]
    ax.plot(SHASH_INCS+cons_intensity[sample],pred_dict[exp_name][RNG_SEED]['pred_cpd_test'][sample,:],
            color=colors[0],
            linewidth=5,
            # clip_on=False,
           )
    ax.axvline(x=true_intensity[sample],
               linestyle='--',               
               color='k',
               alpha=.5,
               linewidth=4,               
               label='BestTrack',
              )
    ax.axvline(x=cons_intensity[sample],
               linestyle='-',               
               alpha=.5,
               linewidth=4,               
               color=colors[1],
               label='Consensus (' + str(cons_error.round(1)) + ')',
              )
    ax.axvline(x=pred_dict[exp_name][RNG_SEED]['pred_med_test'][sample]+cons_intensity[sample],
               linestyle='-',
               color=colors[0],
               linewidth=4,
               alpha=.5,
               label='SHASH3 Median (' + str(ann_error.round(1)) + ')',
              )

    
    details = get_storm_details(df_test, sample)
    ax.set_title(panel_letters[isample] + ' ' + details, fontsize=FS, color='k')
    ax.legend(fontsize=9)
    ax.set_xlabel('intensity (knots)')
    ax.set_xticks(np.arange(0,180,20),np.arange(0,180,20))
    ax.set_yticks(np.arange(0,.16,.02),np.arange(0,.16,.02))    
    ax.set_xlim(0,170)
    ax.set_ylim(-.001,.085)
    format_spines(ax)
    
plt.tight_layout()
# plt.savefig(FIGURE_PATH + 'exampleDistributions_' + model_name + '.png', dpi=dpiFig)    
plt.show()   

In [ ]:
cmap_interp = cm.get_cmap('RdBu_r', N_COLORS)    
value_remap = np.linspace(-15, 15, N_COLORS)
norm = mpl.colors.Normalize(vmin=0, vmax=10)

bins, hist_shash, D_shash, EDp_shash = model_diagnostics.compute_pit('shash',onehot_test, x_data=x_test,model_shash=model)
bins_inc = bins[1]-bins[0]

fig = plt.figure(figsize=(5.5, 6.5), constrained_layout=True)
spec = fig.add_gridspec(4, 3)

sample_count = 0
rng_plot = np.random.default_rng(2175)
plot_samples = rng_plot.choice(np.arange(0,onehot_test.shape[0]),size=6, replace=False)
    
for row in (0,1):
    for col in (0,1,2):
        
        ax = fig.add_subplot(spec[row,col])
        sample = plot_samples[sample_count]
        sample_count += 1        
        
        y = pred_dict[exp_name][RNG_SEED]['pred_cpd_test'][sample,:]
        ax.plot(SHASH_INCS,y,
                color='k',
                linewidth=.5,
               )

        for iperc,perc in enumerate(np.arange(.10,1.10,.10)):
            cumsum = np.cumsum(y)
            ilow = np.argmin(np.abs(cumsum-(perc-.1)))
            ihigh = np.argmin(np.abs(cumsum-(perc)))        

            plt.fill_between(SHASH_INCS[ilow:ihigh+1],y[ilow:ihigh+1], color=cmap_interp(norm(iperc)))

        ax.axvline(x=onehot_test[sample,0],
                   linestyle='--',
                   color='k',
                   alpha=1.,
                   linewidth=2,
                  )
        
        ax.set_xlim(SHASH_INCS[np.argmin(np.abs(cumsum-.001))],SHASH_INCS[np.argmin(np.abs(cumsum-.999))])
        ax.set_ylim(0,.07)

        format_spines(ax)
        ax.set_xticks([], [])
        ax.set_yticks([], [])    
        ax.spines['left'].set_color(None)
    
ax = fig.add_subplot(spec[2:,:])
for ibar in np.arange(0,len(bins)-1):
    plt.bar(bins[ibar]+bins_inc/2,
             hist_shash[0][ibar],
             width=bins_inc*.98,
             color=cmap_interp(norm(ibar)),
             label='SHASH',
            )

ax.axhline(y=.1, 
            linestyle='--',
            color='dimgray', 
            linewidth=2.,
           )
plt.ylim(0,.2)
plt.xticks(bins,np.around(bins,1))
ax = plt.gca()
yticks = np.around(np.arange(0,.25,.05),2)
plt.yticks(yticks,yticks)

plt.xlabel('probability integral transform')
plt.ylabel('probability')
plt.title('PIT Histogram Calculation', fontsize=FS, color='k')

format_spines(ax)
plt.savefig(FIGURE_PATH + 'example_pit_calculation_' + model_name + '.png', dpi=dpiFig)        


Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/Users/eabarnes/opt/anaconda3/envs/env-hurr-tfp/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3823, in atexit_operations
    self.reset(new_session=False)
  File "/Users/eabarnes/opt/anaconda3/envs/env-hurr-tfp/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1429, in reset
    self.history_manager.reset(new_session)
  File "/Users/eabarnes/opt/anaconda3/envs/env-hurr-tfp/lib/python3.9/site-packages/IPython/core/history.py", line 592, in reset
    self.dir_hist[:] = [os.getcwd()]
FileNotFoundError: [Errno 2] No such file or directory


## Compute the probability of RI

In [ ]:
cons_intensity = df_test["VMXC"]
obs_dev_cons_hist, OBS_DEV_BINS = model_diagnostics.compute_clim_errors(
    onehot=np.append(onehot_train[:,0],onehot_val[:,0]), 
    smooth = True,
)
 
x = SHASH_INCS + cons_intensity[sample]
y = pred_dict[exp_name][RNG_SEED]['pred_cpd_test'][sample,:]
x_clim = OBS_DEV_BINS + cons_intensity[sample]
y_clim = obs_dev_cons_hist


vmx0 = 100
vmx_ri = vmx0 + RI_THRESH_DICT[settings['leadtime']]

fig,ax = plt.subplots(1,1)
plt.plot(x,y, 
         color=colors[0],
         linewidth=3,
        )
plt.plot(x_clim,y_clim,
         color=colors[1],
         linewidth=3,
        )
plt.axvline(x=vmx0,
            color='gray',
            label='t=0',
           )
plt.axvline(x=vmx_ri,
            color='gray',
            linestyle='--',
            label='RI threshold',
           )

pr_ri = model_diagnostics.compute_pr_ri(x,y,vmx_ri)*100.
pr_clim_ri = model_diagnostics.compute_pr_ri(x_clim,y_clim,vmx_ri)*100.

plt.title('Probability of RI = ' + str(pr_ri.round()) + '\% ~(' + str(pr_clim_ri.round()) + '\%)')
plt.legend()
plt.xlim(0,180)
ax.set_xlabel('intensity (knots)')
ax.set_xticks(np.arange(0,180,20),np.arange(0,180,20))
ax.set_yticks(np.arange(0,.16,.02),np.arange(0,.16,.02))    
ax.set_xlim(0,170)
ax.set_ylim(-.001,.045)
format_spines(ax)

plt.show()